<a href="https://colab.research.google.com/github/Rula-Islait/Research/blob/main/FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [4]:
class ReverseAttention(layers.Layer):
    def __init__(self, num_heads=8, key_dim=64):
        super(ReverseAttention, self).__init__()
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)

    def call(self, inputs):
        attention_output = self.mha(inputs, inputs)
        return tf.sigmoid(1 - attention_output) + tf.math.tanh(attention_output)

In [5]:
class DynamicAttention(layers.Layer):
    def __init__(self, channels, num_heads=8, key_dim=64):
        super(DynamicAttention, self).__init__()
        self.channels = channels
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.mha = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)
        self.batch_norm = layers.BatchNormalization()
        self.dropout = layers.Dropout(0.3)

    def call(self, inputs):
        attention_output = self.mha(inputs, inputs)
        attention_output = self.batch_norm(attention_output)
        attention_output = self.dropout(attention_output)
        return inputs * attention_output + inputs

In [6]:
class SequentialAttention(layers.Layer):
    def __init__(self, channels, num_heads=8, key_dim=64):
        super(SequentialAttention, self).__init__()
        self.channels = channels
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.mha = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)
        self.conv1 = layers.Conv2D(self.channels, (3, 3), padding='same', activation='relu')
        self.conv2 = layers.Conv2D(self.channels, (3, 3), padding='same', activation='sigmoid')

    def call(self, inputs):
        x = self.conv1(inputs)
        attention_output = self.mha(x, x)
        attention_map = self.conv2(attention_output)
        return inputs + (inputs * attention_map)

In [7]:
class DenseAttention(tf.keras.layers.Layer):
    def __init__(self, channels):
        super(DenseAttention, self).__init__()
        self.channels = channels
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=channels)
        self.pool = tf.keras.layers.GlobalAveragePooling2D()

    def build(self, input_shape):
        self.query_dim = input_shape[-1]

    def call(self, inputs):
        pooled_inputs = self.pool(inputs)
        pooled_inputs = tf.expand_dims(pooled_inputs, 1)
        attention_output = self.mha(pooled_inputs, pooled_inputs)
        return inputs * tf.expand_dims(attention_output, 1) + inputs

In [8]:
def build_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = ReverseAttention(64)(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = DynamicAttention(128)(x)

    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = SequentialAttention(256)(x)

    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = DenseAttention(512)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

In [9]:
input_shape = (28, 28, 1)
model = build_model(input_shape, num_classes)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reverse_attention               │ (None, 14, 14, 64)     │     1,060,928 │
│ (ReverseAttention)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 14, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dynamic_attention               │ (None, 7, 7, 128)      │       264,320 │
│ (DynamicAttention)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 7, 7, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 7, 7, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 7, 7, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_attention            │ (None, 3, 3, 256)      │     1,706,240 │
│ (SequentialAttention)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 3, 3, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 12,320,906 (47.00 MB)

 Trainable params: 12,316,810 (46.98 MB)

 Non-trainable params: 4,096 (16.00 KB)

In [10]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [ ]:
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=80,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/80


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.4885 - loss: 1.5454

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

In [ ]:
from tensorflow.keras.applications import ResNet50, VGG16, MobileNetV2
from tensorflow.keras import layers, models, optimizers

def prepare_data_for_pretrained(x_train, x_test):
    x_train_rgb = tf.image.resize(x_train, [224, 224])
    x_train_rgb = tf.image.grayscale_to_rgb(x_train_rgb)
    x_test_rgb = tf.image.resize(x_test, [224, 224])
    x_test_rgb = tf.image.grayscale_to_rgb(x_test_rgb)
    return x_train_rgb, x_test_rgb

x_train_rgb, x_test_rgb = prepare_data_for_pretrained(x_train, x_test)

def build_transfer_model(base_model, num_classes):
    base_model.trainable = False
    inputs = layers.Input(shape=(224,224,3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model

resnet_model = build_transfer_model(ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3)), num_classes)
mobilenet_model = build_transfer_model(MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3)), num_classes)
vgg_model = build_transfer_model(VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3)), num_classes)

optimizer = optimizers.Adam(learning_rate=0.0001)
for model in [resnet_model, mobilenet_model, vgg_model]:
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
